In [1]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat
from meltcurves import melt_fromdebris_func
from meltcurves import debris_frommelt_func
from spc_split_lists import split_list


debug=False

In [2]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)
        
        
def maskedarray_gt(data, value):
    """ Greater than operation on masked array to avoid warning errors """
    data = np.nan_to_num(data,0)
    data[data > value] = value
    return data


def maskedarray_lt(data, value):
    """ Less than operation on masked array to avoid warning errors """
    data = np.nan_to_num(data,0)
    data[data < value] = value
    return data


def ts_fromdebris_func(h, a, b, c):
    """ estimate surface temperature from debris thickness (h is debris thickness, a and k are coefficients) 
        Hill Equation"""
    return a * h**c / (b**c + h**c)


def debris_fromts_func(ts, a, b, c):
    """ estimate debris thickness from surface temperature (ts is surface temperature, a and k are coefficients) 
        Hill Equation"""
    return (ts * b**c / (a - ts))**(1/c)
        
    
def debris_fromts_maskedarray(ts_raw, a, b, c):
    """ Apply debris_fromts_func to masked array
        includes a mask of maximum values, since Michaelis-Mentin Equation has natural maximum 
    Parameters
    ----------
    ts_raw : np.ma.array
        masked array of the unmodified surface temperature
    Returns
    -------
    hd : np.ma.array 
        masked array of the debris thickness (m)
    """
    ts = ts_raw.copy()
    max_value = ts_fromdebris_func(50, a, b, c)
    debris_thick_ts = np.ma.array(maskedarray_gt(ts.data, max_value), mask=np.ma.getmask(ts))
    debris_thick_ts = np.ma.array(maskedarray_lt(ts.data, 0), mask=np.ma.getmask(ts))
    hd = debris_fromts_func(ts.data, a, b, c)
    return hd

In [3]:
# ===== DETERMINE ALL GLACIERS WITH AND WITHOUT OBSERVATIONS =====
hdopt_prms_fp = debris_prms.output_fp + 'hd_opt_prms/' + debris_prms.roi + '/'

# Glaciers optimized
glac_tsopt_fns = []
rgiid_list_tsopt = []
for i in os.listdir(hdopt_prms_fp):
    if i.endswith('_hdopt_prms.csv'):
        region = int(i.split('.')[0])
        if region in debris_prms.roi_rgidict[debris_prms.roi]:    
            rgiid_list_tsopt.append(i.split('_')[0])            
            glac_tsopt_fns.append(i)
glac_tsopt_fns = sorted(glac_tsopt_fns)
rgiid_list_tsopt = sorted(rgiid_list_tsopt)

main_glac_rgi_tsopt = debris_prms.selectglaciersrgitable(rgiid_list_tsopt)

# All debris-covered glaciers
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_rgiid = sorted([x.split('-')[1] for x in dc_shp.RGIId])
main_glac_rgi_all = debris_prms.selectglaciersrgitable(glac_no=dc_rgiid)

# Merge with debris cover stats
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])
dc_shp.reset_index(inplace=True, drop=True)

# Add debris stats to area
dc_areaperc_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area__1'].values))
dc_area_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_v2'].values))

main_glac_rgi_tsopt['DC_Area_%'] = main_glac_rgi_tsopt.RGIId.map(dc_areaperc_dict).fillna(0)
main_glac_rgi_all['DC_Area_%'] = main_glac_rgi_all.RGIId.map(dc_areaperc_dict).fillna(0)
main_glac_rgi_tsopt['DC_Area_v2'] = main_glac_rgi_tsopt['Area'] * main_glac_rgi_tsopt['DC_Area_%'] / 100
main_glac_rgi_all['DC_Area_v2'] = main_glac_rgi_all['Area'] * main_glac_rgi_all['DC_Area_%'] / 100

# Glaciers lacking optimization
rgiids_missing = set(main_glac_rgi_all.rgino_str.values) - set(main_glac_rgi_tsopt.rgino_str.values)
rgiids_missing = sorted(rgiids_missing)
main_glac_rgi_missing = debris_prms.selectglaciersrgitable(rgiids_missing)
main_glac_rgi_missing

136 glaciers in region 2 are included in this model run: ['00737', '00914', '01104', '01152', '01158', '01161', '01290', '01291', '01297', '01339', '01397', '01441', '01665', '01685', '01727', '01811', '01812', '01922', '01923', '02107', '02348', '02360', '02386', '02432', '02526', '02533', '02550', '02551', '02616', '02686', '02745', '02747', '02752', '02784', '02857', '02894', '02897', '02947', '02948', '02966', '03099', '03102', '03157', '03581', '03597', '03664', '03769', '03922', '03943', '03963'] and more
This study is focusing on 136 glaciers in region [2]
3313 glaciers in region 2 are included in this model run: ['00006', '00016', '00018', '00020', '00039', '00079', '00097', '00098', '00118', '00126', '00134', '00147', '00157', '00173', '00174', '00180', '00208', '00212', '00226', '00227', '00242', '00252', '00255', '00256', '00259', '00280', '00288', '00289', '00291', '00296', '00305', '00317', '00330', '00336', '00343', '00346', '00347', '00354', '00366', '00377', '00378', '0

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float
GlacNo,,,,,,,,,,,,,,,,,,,,
0,5,RGI60-02.00006,-121.21800,49.16680,2,4,0.348,1930,2407,2127,39.8,355,498,0,0,0,20059999,6,02.00006,2.00006
1,15,RGI60-02.00016,-125.74800,49.46180,2,2,0.921,1194,1668,1434,18.1,51,1805,0,0,0,20059999,16,02.00016,2.00016
2,17,RGI60-02.00018,-125.64600,49.48170,2,2,0.623,1378,1741,1583,24.4,352,546,0,0,0,20059999,18,02.00018,2.00018
3,19,RGI60-02.00020,-125.50000,49.48640,2,2,0.307,1133,1429,1286,22.0,9,811,0,0,0,20059999,20,02.00020,2.00020
4,38,RGI60-02.00039,-125.35000,49.54570,2,2,0.328,1658,1898,1824,15.5,89,868,0,0,0,20059999,39,02.00039,2.00039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172,18804,RGI60-02.18805,-109.63833,43.17324,2,5,3.061,3328,4052,3600,19.4,26,2927,0,0,0,19669999,18805,02.18805,2.18805
3173,18805,RGI60-02.18806,-109.69178,43.33450,2,5,2.505,3368,3985,3799,11.1,32,1943,0,0,0,19669999,18806,02.18806,2.18806
3174,18806,RGI60-02.18807,-109.68206,43.31165,2,5,0.577,3817,4011,3961,10.7,118,1276,0,0,0,19669999,18807,02.18807,2.18807


In [4]:
print('\n', debris_prms.roi + ': calibration includes', main_glac_rgi_tsopt.shape[0], 'glaciers covering',
      str(np.round(main_glac_rgi_tsopt['DC_Area_v2'].sum(),1)), 'km2 (' + 
      str(np.round(main_glac_rgi_tsopt['DC_Area_v2'].sum() / main_glac_rgi_all['DC_Area_v2'].sum() * 100,1)) + 
      '%) of the total debris-covered glacier area\n')     


 02: calibration includes 136 glaciers covering 94.2 km2 (21.0%) of the total debris-covered glacier area



In [5]:
# np.where(main_glac_rgi_missing.rgino_str == '18.02342')

In [ ]:
extrap_uncalibrated_glaciers = True
overwrite_hd = False

hd_fp = debris_prms.hd_fp + 'extrap/'
if not os.path.exists(hd_fp):
    os.makedirs(hd_fp)
    
mf_fp = hd_fp + 'meltfactor/'
if not os.path.exists(mf_fp):
    os.makedirs(mf_fp)
    
fig_extrap = debris_prms.output_fig_fp + debris_prms.roi + '/' + 'extrap/'
if not os.path.exists(fig_extrap):
    os.makedirs(fig_extrap)

if extrap_uncalibrated_glaciers:
    
    # ===== NEAREST GLACIERS WITH DATA =====
    n_glac_nearest = 100
    if n_glac_nearest > main_glac_rgi_tsopt.shape[0]:
        n_glac_nearest = main_glac_rgi_tsopt.shape[0]

    nearest_dict = {}
    for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_missing.index.values[755]]):
        glac_str = main_glac_rgi_missing.loc[glac_idx, 'rgino_str']
        if glac_idx%500 == 0:
            print(glac_idx, glac_str)
        latlon_dist = (((main_glac_rgi_tsopt['CenLat'].values - main_glac_rgi_missing['CenLat'].values[glac_idx])**2 + 
                        (main_glac_rgi_tsopt['CenLon'].values - main_glac_rgi_missing['CenLon'].values[glac_idx])**2)**0.5)

        latlon_nearidx_list = np.argsort(latlon_dist)[0:n_glac_nearest]
        rgiid_nearest_list = list(main_glac_rgi_tsopt.loc[latlon_nearidx_list,'rgino_str'].values)

        nearest_dict[glac_str] = rgiid_nearest_list

    # Ts filenames
    ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)
    
    for nglac, glac_idx in enumerate(main_glac_rgi_missing.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_missing.index.values[755]]):
        glac_str = main_glac_rgi_missing.loc[glac_idx, 'rgino_str']
        rgiid = main_glac_rgi_missing.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if int(region) < 10:
            glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
        else:
            glac_str_noleadzero = glac_str

        # Ts filename
        ts_fn_idx = np.where(ts_fns_df['RGIId'].values == rgiid)[0][0]
        ts_fn = ts_fns_df.loc[ts_fn_idx,'ts_fullfn']

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
        gf = create_glacfeat(thick_dir, thick_fn)
        
        # Only process new ones
        hd_fn = debris_prms.hd_fn_sample.replace('XXXX',gf.glacnum).replace('.tif','_extrap.tif')
        
        if not os.path.exists(hd_fp + hd_fn) or overwrite_hd:
            
            print(glac_idx, glac_str)
        
            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if not os.path.exists(dc_shp_proj_fn):
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + 
                                            debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + 
                                                    str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()

            # Add layers
            gf.add_layers(dc_shp_lyr, gf_add_ts=True, ts_fn=ts_fn, gf_add_slope_aspect=False)

            # ===== PLOTS =====
            if debug:
                # DEM
                var_full2plot = gf.z1.copy()
                clim = malib.calcperc(var_full2plot, (2,98))
                plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                # Surface temperature
                var_full2plot = gf.ts.copy()
                clim = malib.calcperc(var_full2plot, (2,98))
                plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
                # Surface temperature (debris-covered)
                var_full2plot = gf.ts.copy()
                var_full2plot.mask = gf.dc_mask
                clim = malib.calcperc(var_full2plot, (2,98))
                plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
            
            # ===== ESTIMATE DEBRIS THICKNESS FOR EACH GLACIER INDIVIDUALLY =====
            # Load parameters from nearest neighbor
            rgiid_nearest_list = nearest_dict[main_glac_rgi_missing.loc[glac_idx, 'rgino_str']]

            n_nearest = 0
            n_success = 0
            min_n_nearest = 10
            hd_ts_list = []
            mf_list = []
            while n_nearest < n_glac_nearest and n_success < min_n_nearest:
                rgi_str_nearest = rgiid_nearest_list[n_nearest]
                if rgi_str_nearest.startswith('0'):
                    rgi_str_nearest = rgi_str_nearest[1:]
                if debug:
                    print('rgi nearest:', rgi_str_nearest)

                # Load parameters
                df_opt_fn = rgi_str_nearest + '_hdopt_prms.csv'
                df_opt = pd.read_csv(hdopt_prms_fp + df_opt_fn)
                melt_2cm = df_opt.loc[0,'melt_mwea_2cm']
                melt_cleanice = df_opt.loc[0,'melt_mwea_clean']
                func_coeff = [df_opt.loc[0,'b0'], df_opt.loc[0,'k']]
                func_coeff_ts = [df_opt.loc[0,'a'], df_opt.loc[0,'b'], df_opt.loc[0,'c']]

                # Estimate debris thickness
                hd_array = debris_fromts_maskedarray(gf.ts, func_coeff_ts[0], func_coeff_ts[1], func_coeff_ts[2])
                hd_array[hd_array>debris_prms.hd_max] = debris_prms.hd_max
                hd_ma = np.ma.array(hd_array, mask=gf.dc_mask)
                hd_ma_median = np.median(hd_ma.compressed())
                
                if debug:
                    print('hd median:', np.round(hd_ma_median,2))

                # Only include estimates if they are plausible
                if hd_ma_median > 0 and hd_ma_median < debris_prms.hd_max:
                    hd_ts_list.append(hd_ma)
                    n_success += 1

                    # Melt factor
                    mf_array = melt_fromdebris_func(hd_array, func_coeff[0], func_coeff[1]) / melt_cleanice
                    mf_ma = np.ma.array(maskedarray_gt(mf_array, melt_2cm / melt_cleanice), mask=np.ma.getmask(gf.ts))
                    # Linearly interpolate between 0 cm and 2 cm for the melt rate
                    def meltfactor_0to2cm_adjustment(mf, melt_clean, melt_2cm, hd):
                        """ Linearly interpolate melt factors between 0 and 2 cm 
                            based on clean ice and 2 cm sub-debris melt """
                        mf = np.nan_to_num(mf,0)
                        mf[(hd >= 0) & (hd < 0.02)] = (
                            1 + hd[(hd >= 0) & (hd < 0.02)] / 0.02 * (melt_2cm - melt_clean) / melt_clean)
                        return mf
                    mf_ma = np.ma.array(meltfactor_0to2cm_adjustment(mf_ma.data.copy(), melt_cleanice, melt_2cm, 
                                                                     hd_ma.data), mask=np.ma.getmask(gf.ts))
                    mf_list.append(mf_ma)
                        
                n_nearest += 1

            # ===== ESTIMATE DEBRIS THICKNESS FROM ALL COMBINATIONS =====
            if len(hd_ts_list) > 0:
                # DEBRIS THICKNESS based on median of the plausible nearest values
                hd_ts_all = np.ma.array(hd_ts_list)
                hd_ts_med = np.median(hd_ts_all, axis=0)
                gf.debris_thick_ts = hd_ts_med

                if debug:
                    close_fig=False
                else:
                    close_fig=True
                    
                # Debris thickness
                var_full2plot = gf.debris_thick_ts.copy()
                clim = (0,1)
                plot_array(var_full2plot, clim, [gf.glacnum + ' hd (from ts)'], 'inferno', 'hd (m)', 
                           fn=fig_extrap + gf.feat_fn +'_hd_ts.png', close_fig=close_fig)
                
                # MELT FACTOR based on median of the plausible nearest values
                mf_all = np.ma.array(mf_list)
                mf_med = np.median(mf_all, axis=0)
                gf.meltfactor_ts = mf_med
                
                # Melt factor
                var_full2plot = gf.meltfactor_ts.copy()
                clim = (0,1.25)
                plot_array(var_full2plot, clim, [gf.glacnum + ' melt factor'], 'inferno', 'melt factor (-)', 
                           fn=fig_extrap + gf.feat_fn +'_mf.png', close_fig=close_fig)
                
                
                # ===== EXPORT DEBRIS THICKNESS AND MELT FACTOR TIFS ===== 
                # Debris thickness
                gf.debris_thick_ts.mask = gf.dc_mask
                iolib.writeGTiff(gf.debris_thick_ts, hd_fp + hd_fn, gf.ds_dict['z1'])
                
                # Melt factor
                gf.meltfactor_ts.mask = gf.dc_mask
                mf_fn = debris_prms.mf_fn_sample.replace('XXXX',gf.glacnum).replace('.tif','_extrap.tif')
                iolib.writeGTiff(gf.meltfactor_ts, mf_fp + mf_fn, gf.ds_dict['z1'])

                
                # ===== EXPORT THE BINNED DEBRIS THICKNESS AND MELT FACTOR =====
                # Output debris thickness
                outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                hd_extrap_bin_fp = debris_prms.mb_binned_fp_wdebris_hdts + '../_wdebris_hdts_extrap/'
                if not os.path.exists(hd_extrap_bin_fp):
                    os.makedirs(hd_extrap_bin_fp)
                outbins_df.to_csv(hd_extrap_bin_fp + glac_str + '_mb_bins_hdts_extrap.csv', index=False)
                
            else:
                troubleshoot_fp = (debris_prms.output_fp + 'errors/extrap_failed/' + debris_prms.roi + '/')
                if not os.path.exists(troubleshoot_fp):
                    os.makedirs(troubleshoot_fp)
                txt_fn_extrapfail = glac_str + "-extrap_failed.txt"
                with open(troubleshoot_fp + txt_fn_extrapfail, "w") as text_file:
                    text_file.write(glac_str + ' failed to any reasonable extrapolation estimates')



0 02.00006
500 02.03030
1000 02.05800
1500 02.09758
2000 02.12922
2500 02.13950
3000 02.17862
0 02.00006


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/numpy/core/fromnumeric.py:745: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


1 02.00016
2 02.00018
3 02.00020
4 02.00039
5 02.00079
6 02.00097
7 02.00098
8 02.00118
9 02.00126
10 02.00134
11 02.00147
12 02.00157
13 02.00173
14 02.00174
15 02.00180
16 02.00208
17 02.00212
18 02.00226
19 02.00227
20 02.00242
21 02.00252
22 02.00255
23 02.00256
24 02.00259
25 02.00280
26 02.00288
27 02.00289
28 02.00291
29 02.00296
30 02.00305
31 02.00317
32 02.00330
33 02.00336
34 02.00343
35 02.00346
36 02.00347
37 02.00354
38 02.00366


In [ ]:
print('\nDONE!\n')

In [ ]:
print('TO-DO LIST:\n  - Mosaic extrapolated and calibrated glaciers into 1-degree pixels')